In [ ]:
import pypsa
import pandas as pd
import xarray as xr

n = pypsa.Network("marginal_system_value/input/elec_s_37_ec_lcopt_Co2L0.0-24H.nc")

print(n)

In [ ]:
n.generators['p_nom_extendable'] = True

carriers_to_modify_1 = ["ror"]

n.generators.loc[n.generators['carrier'].isin(carriers_to_modify_1), 'p_nom_max'] = n.generators['p_nom']

target_generators = n.generators[
    n.generators['carrier'].isin(['biomass', 'nuclear'])
]

n.generators.loc[target_generators.index, 'p_nom_min'] = target_generators['p_nom']

n.storage_units['p_nom_extendable'] = True

n.storage_units['p_nom_max'] = n.storage_units['p_nom']

carriers_to_modify = ["oil", "lignite", "coal", "CCGT", "OCGT"]

n.generators.loc[n.generators['carrier'].isin(carriers_to_modify), 'p_nom_min'] = 0

In [ ]:
n.add("Carrier", "SBSP", co2_emissions=0.0)

ac_buses = n.buses[n.buses['carrier'] == 'AC']

for bus in ac_buses.index:
    n.add("Generator", f"SBSP Generator at {bus}", 
          bus=bus,  
          carrier="SBSP",  
          p_nom_min=0.0,  # MW
          marginal_cost=0.0,  # EUR/MWh
          capital_cost=200000,  # EUR/MW
          p_nom_extendable=True  
         )

sbsp_limits = xr.open_dataset("marginal_system_value/input/sbsp_rd2_profile_2050.nc")
effective_power = sbsp_limits["Effective Power (MW)"].to_dataframe()

effective_power.index = pd.to_datetime(effective_power.index)

daily_effective_power = effective_power.resample('24H').mean()

for gen_id in n.generators.index:
    if n.generators.loc[gen_id, 'carrier'] == 'SBSP':

        for timestamp in daily_effective_power.index:
            n.generators_t.p_max_pu.loc[timestamp, gen_id] = daily_effective_power.loc[
                timestamp, 'Effective Power (MW)']

In [ ]:
snapshots_to_optimize = n.snapshots[:366]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

n.export_to_netcdf('marginal_system_value/results/2050-rd2-200000-24h/optimized_2050_rd2_200000_network.nc')

supply_data = n.statistics.supply(comps=["Generator", "StorageUnit", "Store"], aggregate_time=False)

supply_data = supply_data.droplevel(0).T

print("Available columns (energy types):")
print(supply_data.columns)

supply_data.to_csv('marginal_system_value/results/2050-rd2-200000-24h/2050_middle_rd2_200000_hourly_energy_supply.csv', index=True)

In [ ]:
# CapEx
total_capex = n.statistics.capex()

# OpEx
total_opex = n.statistics.opex(aggregate_time="sum")

# total_cost
total_cost = total_capex + total_opex

with open("marginal_system_value/results/2050-rd2-200000-24h/total_cost_summary_rd2_200000.txt", "w") as f:
    f.write(f"Total CapEx: {total_capex}\n")
    f.write(f"Total OpEx: {total_opex}\n")
    f.write(f"Total Cost (CapEx + OpEx): {total_cost}\n")

print("Results saved to total_cost_summary.txt")